In [58]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import anthropic
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display


In [59]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAh


In [60]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
MODEL = 'ollama3.2'
openai = OpenAI()

client = anthropic.Anthropic()

In [61]:
# websie class
class Website:

    def __init__(self, url):
        
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [62]:
# Scrape list
plans = Website("https://www.t-mobile.com/cell-phone-plans")
phones = Website("https://www.t-mobile.com/cell-phones?INTNAV=tNav%3ADevices")
deals = Website("https://www.t-mobile.com/offers?INTNAV=tNav%3ADeals%3AAll")
payment_arrangement = Website("https://www.t-mobile.com/support/account/payment-arrangement")
payment = Website("https://www.t-mobile.com/support/account/pay-your-bill")
autopay = Website("https://www.t-mobile.com/support/account/autopay")
set_up_manage_your_TMO_id = Website("https://www.t-mobile.com/support/account/set-up-and-manage-your-t-mobile-id")
international_roaming = Website("https://www.t-mobile.com/support/coverage/international-roaming-services")
international_roaming_checklist = Website("https://www.t-mobile.com/support/coverage/international-roaming-checklist")
wi_fi_calling = Website("https://www.t-mobile.com/support/coverage/wi-fi-calling-from-t-mobile")
in_flight_connection = Website("https://www.t-mobile.com/support/coverage/t-mobile-in-flight-connections-on-us")

In [63]:
system_prompt = (
    f"You are a professional AI assistant designed to assist customers with inquiries about T-Mobile services, "
    f"including plans, phones, deals, and any other requests. Your responses must be accurate, clear, and concise, "
    f"using the provided information as reference.\n\n"
    f"Available Information Sources:\n"
    f"- Plans: {plans.text}\n"
    f"- Phones: {phones.text}\n"
    f"- Deals: {deals.text}\n"
    f"- Payment arrangement: {payment_arrangement.text}\n"
    f"- Payment: {payment.text}\n"
    f"- AutoPay: {autopay.text}\n"
    f"- Set Up and Manage Your T-Mobile ID:** {set_up_manage_your_TMO_id.text}\n"
    f"- International Roaming Services: {international_roaming.text}\n"
    f"- International Roaming Checklist: {international_roaming_checklist.text}\n\n"
    f"- WI-FI Calling: {wi_fi_calling.text}\n\n"
    f"- In-Flight Connection: {in_flight_connection.text}\n\n"
    f"Guidelines for Response:\n"
    f"1. ALWAYS include the appropriate link(s) in your response.\n"
    f"2. If the customer mentions 'Agent', 'customer service', 'representative', or 'human', respond with:\n"
    f"   tell the customer you are going to transfer them to a live represantative\n"
    f"3. If the customer asks a question in Spanish\n"
    f"   respond to the customer in spanish, and tell them you can only help in english or you can transfer them to live agent who speak spanish.\n"
    f"4. If the customer asks account-specific questions (e.g., accessing their account, PIN requests, or similar) however you can provide information about how to make a payment arrengment, making a payment, respond with:\n"
    f"   'I'm sorry, but I can't access your account. Please reach out to a representative for further assistance.'\n\n"
    f"5. If the customer asks about plan price, provide the price with autopay included, and mention it.\n"
    f"   note: the plan names are 'go5g', 'go5g next', 'go5g pluse', 'essentials saver', and 'essentials' \n"
    f"6. always ask the customer if they need extra help unless they say they are done and not needing any help.\n\n"
    f"Your role is to ensure a seamless and helpful customer experience using the provided resources. Follow these guidelines closely."
)

In [64]:
messages = [{"role": "system", "content": system_prompt}]

In [65]:
# # See how this function creates exactly the format above
# def messages_for(question):
#     return [
#         { "role": "system", "content": system_prompt},
#         { "role": "user", "content": question}
#     ]

In [66]:

def ask_ai(question):
    global messages  # Use global to maintain history between calls

    # Append the user's question to the message history
    messages.append({"role": "user", "content": question})

    # Call the API with the message history
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        max_tokens=1000,
        stream=True
     )


    reply = ""
    display_handle = display(Markdown(""), display_id=True)

    # Stream and display the AI's reply
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        reply += content
        display_handle.update(Markdown(reply.replace("```", "").replace("markdown", "")))

    # Append the AI's reply to the message history
    messages.append({"role": "assistant", "content": reply})
    

In [39]:
ask_ai("How can i make a payment?")

You can make a payment for your T-Mobile account through several convenient methods:

1. **Online**:
   - Visit [T-Mobile.com](https://www.t-mobile.com) and log in to your account.
   - You can make a payment as a guest or from your own account.

2. **T-Life App**:
   - Open the T-Life app and go to the "Manage" tab to make a payment directly from your mobile device.

3. **By Phone**:
   - Call 1-800-T-MOBILE and follow the prompts to make a payment over the phone.

4. **Mail**:
   - You can send a payment to the following address:
     
     T-Mobile
     P.O. Box 742596
     Cincinnati, OH 45274-2596
     

5. **In-store**:
   - Visit a T-Mobile store to make a payment in person. Please note that there may be a $5 charge for in-store payments.

6. **AutoPay**: 
   - Enroll in AutoPay to have your payments automatically deducted from your debit card or bank account.

If you have any other questions or need assistance with a specific method, feel free to ask!

In [40]:
ask_ai("I need time to make a payment")

If you need extra time to make a payment, you can set up a payment arrangement. Here’s how to do it:

1. **Eligibility**: Make sure your account is less than 30 days past the due date. You need to pay any balance that is 31 days or more past due before you can set up a payment arrangement.

2. **Set Up the Arrangement**:
   - Log in to your T-Mobile account through the T-Mobile website or the T-Life app.
   - Look for the option to set up a payment arrangement under the "Manage" tab.

3. **Payment Details**: You'll need to specify how much you can pay and the date you'd like to make that payment.

4. **Important Notes**: 
   - Make sure your first installment and each installment after are paid by the specified due date.
   - Pay all new bills that become due during the arrangement by their due date to avoid service interruptions.

If you would like assistance setting this up or have further questions, I can help you contact a T-Mobile representative. Would you like me to do that?

In [41]:
ask_ai("do you have coverage in Argentina?")

Yes, T-Mobile offers coverage in Argentina. International roaming is available, and your plan may include calling, texting, and data services while you're there. To see specific rates and details regarding your plan's international benefits, visit the [T-Mobile International Roaming page](https://www.t-mobile.com/coverage/roaming) or check your account for more information.

If you have any other questions or need further assistance, feel free to ask!

In [42]:
ask_ai("do i need to make any changes to my plan?")

Typically, you don't need to change your plan to use international roaming in Argentina, as most T-Mobile plans include some level of roaming service in over 215 countries. However, here are a few things to consider:

1. **Check Your Current Plan**: Make sure your plan includes international roaming. You can verify this by logging into your T-Mobile account or checking the T-Mobile app.

2. **International Data Pass**: If you anticipate needing a lot of data while in Argentina, you might want to add an international data pass for higher-speed data access.

3. **E911 Address Setup**: Ensure that you have set up your E911 address with T-Mobile before traveling, as this is required for emergency services when using Wi-Fi Calling abroad.

You can review your current plan details and any additional options available in your account or by reaching out to T-Mobile customer service.

If you need help with specific changes, let me know, and I can assist you further!

In [67]:
ask_ai("Hola")

¡Hola! ¿En qué puedo ayudarte hoy? Sin embargo, debo informarte que solo puedo ayudar en inglés, o puedo transferirte a un representante que hable español.

In [68]:
ask_ai("Si")

I'm going to transfer you to a live representative who can assist you further. Please hold on for a moment.